In [ ]:
import os
import csv
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
BASE_URL = "https://api.covidtracking.com/v1"

# Define all endpoints
endpoints = {
    "us_daily": "/us/daily.json",
    "us_current": "/us/current.json",
    "states_daily": "/states/daily.json",
    "states_current": "/states/current.json",
    "states_info": "/states/info.json"
}

# Fetch and save data for each endpoint
for name, endpoint in endpoints.items():
    url = BASE_URL + endpoint
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data)
        df.to_csv(f"{name}.csv", index=False)
        print(f"✅ Successfully saved {name}.csv")
    else:
        print(f"❌ Failed to fetch {name}. Status code: {response.status_code}")

✅ Successfully saved us_daily.csv
✅ Successfully saved us_current.csv
✅ Successfully saved states_daily.csv
✅ Successfully saved states_current.csv
✅ Successfully saved states_info.csv
